In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/home-credit-default-risk'

In [ ]:
import pandas as pd 

import warnings
warnings.filterwarnings('ignore')

In [ ]:
app_train = pd.read_csv(path + '/application_train.csv')
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print('Training data shape: ', app_train.shape)

Training data shape:  (307511, 122)


In [ ]:
bureau = pd.read_csv(path + '/bureau.csv')

In [ ]:
bureau = bureau[:-1]

In [ ]:
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716422,375755,5057742,Closed,currency 1,-2648,0,31129.0,-189.0,NaN,0,202500.00,0.0,NaN,0.0,Credit card,-109,NaN
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN


In [ ]:
bureau_balance = pd.read_csv(path + '/bureau_balance.csv')

In [ ]:
bureau_balance

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


In [ ]:
bureau_id = pd.DataFrame(columns = ['SK_ID_CURR'])
bureau_id['SK_ID_CURR'] = bureau['SK_ID_CURR'].unique()
bureau_id

,SK_ID_CURR
0,215354
1,162297
2,402440
3,238881
4,222183
...,...
305806,207190
305807,324956
305808,448157
305809,345866


In [ ]:
month_num = bureau_balance[bureau_balance["STATUS"] != 'X'].groupby('SK_ID_BUREAU').count()['MONTHS_BALANCE']
closed_num = bureau_balance[bureau_balance["STATUS"] == 'C'].groupby('SK_ID_BUREAU').count()['MONTHS_BALANCE']
closed_percent = pd.DataFrame(closed_num / month_num).fillna(0)
not_given = bureau_balance[bureau_balance["STATUS"] == 'X'].groupby('SK_ID_BUREAU').count()['MONTHS_BALANCE'] / month_num
not_given = pd.DataFrame(not_given[not_given == 1])
a = pd.concat([closed_percent, not_given]).reset_index().groupby('SK_ID_BUREAU').count().values
closed_percent[a == 2] = -1

In [ ]:
dpd = pd.DataFrame(index = bureau_balance.groupby('SK_ID_BUREAU').count().index)
dpd['0'] = 0
dpd['1'] = 0
dpd['2'] = 0
dpd['3'] = 0
dpd['4'] = 0
dpd['5'] = 0

dpd['0'] += bureau_balance[bureau_balance["STATUS"] == '0'].groupby('SK_ID_BUREAU').count()['STATUS']
dpd['1'] += bureau_balance[bureau_balance["STATUS"] == '1'].groupby('SK_ID_BUREAU').count()['STATUS']
dpd['2'] += bureau_balance[bureau_balance["STATUS"] == '2'].groupby('SK_ID_BUREAU').count()['STATUS']
dpd['3'] += bureau_balance[bureau_balance["STATUS"] == '3'].groupby('SK_ID_BUREAU').count()['STATUS']
dpd['4'] += bureau_balance[bureau_balance["STATUS"] == '4'].groupby('SK_ID_BUREAU').count()['STATUS']
dpd['5'] += bureau_balance[bureau_balance["STATUS"] == '5'].groupby('SK_ID_BUREAU').count()['STATUS']

dpd = dpd.fillna(0)
dpd['avg'] = (dpd['0'] + dpd['1'] * 2 + dpd['2'] * 3 + dpd['3'] * 4 + dpd['4'] * 5 + dpd['5'] * 6) / month_num
dpd = pd.DataFrame(dpd['avg'])

dpd.columns = ['MONTHS_BALANCE']
a = pd.concat([dpd, not_given]).reset_index().groupby('SK_ID_BUREAU').count()
dpd[a == 2] = -1

In [ ]:
t = pd.DataFrame(index = bureau['SK_ID_BUREAU'])
t['closed_percent'] = 0
t['closed_percent'] += closed_percent['MONTHS_BALANCE']
bureau = bureau.merge(t.reset_index(), on='SK_ID_BUREAU')

In [ ]:
t = pd.DataFrame(index = bureau['SK_ID_BUREAU'])
t['dpd'] = 0
t['dpd'] += dpd['MONTHS_BALANCE']

bureau = bureau.merge(t.reset_index(), on='SK_ID_BUREAU')

In [ ]:
long_num = bureau_balance[bureau_balance["STATUS"] == '5'].groupby('SK_ID_BUREAU').count()['MONTHS_BALANCE']
long_percent = pd.DataFrame(long_num / month_num).fillna(0)
a = pd.concat([long_percent, not_given]).reset_index().groupby('SK_ID_BUREAU').count().values
long_percent[a == 2] = -1
t = pd.DataFrame(index = bureau['SK_ID_BUREAU'])
t['long_percent'] = 0
t['long_percent'] += long_percent['MONTHS_BALANCE']
bureau = bureau.merge(t.reset_index(), on='SK_ID_BUREAU')

In [ ]:
good_num = bureau_balance[(bureau_balance["STATUS"] == 'C') | (bureau_balance["STATUS"] == '0') |
                          (bureau_balance["STATUS"] == '1')].groupby('SK_ID_BUREAU').count()['MONTHS_BALANCE']
good_percent = pd.DataFrame(good_num / month_num).fillna(0)
a = pd.concat([good_percent, not_given]).reset_index().groupby('SK_ID_BUREAU').count().values
good_percent[a == 2] = -1

t = pd.DataFrame(index = bureau['SK_ID_BUREAU'])
t['good_percent'] = 0
t['good_percent'] += good_percent['MONTHS_BALANCE']
bureau = bureau.merge(t.reset_index(), on='SK_ID_BUREAU')

In [ ]:
import numpy as np
last_months = bureau_balance.drop_duplicates(subset=['SK_ID_BUREAU'])
last_closed = pd.DataFrame(np.vectorize(lambda x: 1 if x == 'C' else (-1 if x == 'X' else 0))(last_months['STATUS']))
last_closed.index = last_months['SK_ID_BUREAU'].values

t = pd.DataFrame(index = bureau['SK_ID_BUREAU'])
t['last_closed'] = 0
t['last_closed'] += last_closed[0]
bureau = bureau.merge(t.reset_index(), on='SK_ID_BUREAU')

closed_percent - доля месяцев в которые по данной заявке была выплачена вся сумма и -1, если вообще нет данных по статусу этой заявки

dpd - среднее количество месяцев, на которое клиент просрочивал выплаты и -1, если вообще нет данных по статусу этой заявки

long_percent - доля месяцев, в которые клиент просрочил выплаты на 120+ дней

good_percent - доля месяцев, в которые задержка была не более 2 месяцев

last_closed - был ли погашен кредит в последний месяц из тех, по которым есть данные

In [ ]:
bureau['fact_credit_duration'] = bureau['DAYS_ENDDATE_FACT'] - bureau['DAYS_CREDIT']
bureau['delay'] = bureau['DAYS_ENDDATE_FACT']  - bureau['DAYS_CREDIT_ENDDATE']

fact_credit_duration - сколько дней выплачивался кредит (для закрытых кредитов, иначе None)

delay - задержка по выплате кредита (отрицательная, если кредит был погашен досрочно) (для закрытых кредитов, иначе None)

In [ ]:
bureau['CREDIT_TYPE'].unique()

array(['Consumer credit', 'Credit card', 'Mortgage', 'Car loan',
       'Microloan', 'Loan for working capital replenishment',
       'Loan for business development', 'Real estate loan',
       'Unknown type of loan', 'Another type of loan',
       'Cash loan (non-earmarked)', 'Loan for the purchase of equipment',
       'Mobile operator loan', 'Interbank credit',
       'Loan for purchase of shares (margin lending)'], dtype=object)

In [ ]:
bureau[bureau['CREDIT_TYPE'] == 'Real estate loan']['CREDIT_TYPE'] = 'Mortgage'

In [ ]:
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,closed_percent,dpd,long_percent,good_percent,last_closed,fact_credit_duration,delay
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,NaN,NaN,NaN,NaN,344.0,0.0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716422,375755,5057742,Closed,currency 1,-2648,0,31129.0,-189.0,NaN,0,202500.00,0.0,NaN,0.0,Credit card,-109,NaN,NaN,NaN,NaN,NaN,NaN,2459.0,-31318.0
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN,NaN,NaN,NaN,NaN,NaN,155.0,-60.0
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN,NaN,NaN,NaN,NaN,NaN,839.0,658.0


In [ ]:
def count_apps(index, data, column, credit_type, time):
  num = pd.DataFrame(index = index, columns=['0'])
  if credit_type == 'all':
    num['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_CREDIT'] >= -365*time].groupby('SK_ID_CURR')[column].count()
  else:
    num['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_CREDIT'] >= -365*time][data['CREDIT_TYPE'] == credit_type].groupby('SK_ID_CURR')[column].count()
  return num['0'].fillna(0)

def sum_apps(index, data, column, credit_type, time):
  sum = pd.DataFrame(index = index, columns=['0'])
  if credit_type == 'all':
    sum['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_CREDIT'] >= -365*time].groupby('SK_ID_CURR')[column].sum()
  else:
    sum['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_CREDIT'] >= -365*time][data['CREDIT_TYPE'] == credit_type].groupby('SK_ID_CURR')[column].sum()
  return sum['0'].fillna(0)

def mean_apps(index, data, column, credit_type, time):
  mean = pd.DataFrame(index = index, columns=['0'])
  if credit_type == 'all':
    mean['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_CREDIT'] >= -365*time].groupby('SK_ID_CURR')[column].mean()
  else:
    mean['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_CREDIT'] >= -365*time][data['CREDIT_TYPE'] == credit_type].groupby('SK_ID_CURR')[column].mean()
  return mean['0'].fillna(0)

def max_apps(index, data, column, credit_type, time):
  max = pd.DataFrame(index = index, columns=['0'])
  if credit_type == 'all':
    max['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_CREDIT'] >= -365*time].groupby('SK_ID_CURR')[column].max()
  else:
    max['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_CREDIT'] >= -365*time][data['CREDIT_TYPE'] == credit_type].groupby('SK_ID_CURR')[column].max()
  return max['0'].fillna(0)

In [ ]:
app_train_b = app_train.merge(bureau_id, on='SK_ID_CURR').set_index(['SK_ID_CURR'])

In [ ]:
#count
time = [1, 3, 1000]
types = ['Credit card','Consumer credit','Mortgage','Car loan', 'all']
index = app_train_b.index
for t in time:
  for ty in types:
    if t != 1000:
      app_train_b['b_' + ty + '_num_' + str(t)] = count_apps(index, bureau, 'SK_ID_BUREAU', ty, t)
      app_train_b['b_overdue_num_' + ty + '_' + str(t)] = count_apps(index, bureau[bureau['AMT_CREDIT_SUM_OVERDUE'] > 0], 'SK_ID_BUREAU', ty, t)
    else:
      app_train_b['b_' + ty + '_num'] = count_apps(index, bureau, 'SK_ID_BUREAU', ty, t)
      app_train_b['b_overdue_num_' + ty] = count_apps(index, bureau[bureau['AMT_CREDIT_SUM_OVERDUE'] > 0], 'SK_ID_BUREAU', ty, t)
  if t != 1000:
    app_train_b[['b_Microloan_num_' + str(t)]] = count_apps(index, bureau, 'SK_ID_BUREAU', 'Microloan', t)
  else:
    app_train_b[['b_Microloan_num']] = count_apps(index, bureau, 'SK_ID_BUREAU', 'Microloan', t)
    app_train_b[['b_Microloan_active_num' + str(t)]] = count_apps(index, bureau[bureau['CREDIT_ACTIVE'] == 'Active'], 'SK_ID_BUREAU', 'Microloan', t)

for ty in types:
    app_train_b['b_active_' + ty + '_num'] = count_apps(index, bureau[bureau['CREDIT_ACTIVE'] == 'Active'], 'SK_ID_BUREAU', ty, 1000)
    app_train_b['b_closed_' + ty + '_num'] = count_apps(index, bureau[bureau['CREDIT_ACTIVE'] == 'Closed'], 'SK_ID_BUREAU', ty, 1000)
    app_train_b['b_now_overdue_num_' + ty] = count_apps(index, bureau[bureau['CREDIT_DAY_OVERDUE'] > 0], 'SK_ID_BUREAU', ty, 1000)
    app_train_b['b_prolong_num_' + ty] = count_apps(index, bureau[bureau['CNT_CREDIT_PROLONG'] > 0], 'SK_ID_BUREAU', ty, 1000)
    if ty != 'Credit card':
      app_train_b['b_late_num_' + ty] = count_apps(index, bureau[(bureau['delay'] > 0) & (bureau['CREDIT_TYPE'] != 'Credit card')], 'SK_ID_BUREAU', ty, 1000)
      app_train_b['b_early_num_' + ty] = count_apps(index, bureau[(bureau['delay']< 0) & (bureau['CREDIT_TYPE'] != 'Credit card')], 'SK_ID_BUREAU', ty, 1000)

app_train_b['b_debt_num'] = count_apps(index, bureau[bureau['AMT_CREDIT_SUM_DEBT'] > 0], 'SK_ID_BUREAU', 'all', 1000)

In [ ]:
#sum
for t in time:
  for ty in types:
    if t != 1000:
      if ty != 'Mortgage':
        app_train_b['b_' + ty + '_sum_' + str(t)] = sum_apps(index, bureau, 'AMT_CREDIT_SUM', ty, t)
    else:
      app_train_b['b_' + ty + '_sum'] = sum_apps(index, bureau, 'AMT_CREDIT_SUM', ty, t)

app_train_b['b_debt_sum'] = sum_apps(index, bureau[bureau['AMT_CREDIT_SUM_DEBT'] > 0], 'AMT_CREDIT_SUM_DEBT', 'all', 1000)
app_train_b['b_left_sum'] = sum_apps(index, bureau[bureau['AMT_CREDIT_SUM'] > 0], 'AMT_CREDIT_SUM', 'all', 1000)

In [ ]:
#mean
for t in time:
  for ty in types:
    if t != 1000:
      if ty != 'Mortgage':
        app_train_b['b_' + ty + '_avg_' + str(t)] = mean_apps(index, bureau, 'AMT_CREDIT_SUM', ty, t)
      app_train_b['b_overdue_avg_' + ty + '_' + str(t)] = mean_apps(index, bureau[bureau['AMT_CREDIT_SUM_OVERDUE'] > 0], 'AMT_CREDIT_SUM_OVERDUE', ty, t)
    else:
      app_train_b['b_' + ty + '_avg'] = mean_apps(index, bureau, 'AMT_CREDIT_SUM', ty, t)
      app_train_b['b_overdue_avg_' + ty] = mean_apps(index, bureau[bureau['AMT_CREDIT_SUM_OVERDUE'] > 0], 'AMT_CREDIT_SUM_OVERDUE', ty, t)

for ty in types:
  app_train_b['b_active_' + ty + '_dur_avg'] = mean_apps(index, bureau[bureau['DAYS_CREDIT_ENDDATE'] > 0], 'DAYS_CREDIT_ENDDATE', ty, 1000)
  app_train_b['b_start_' + ty + '_avg'] = mean_apps(index, bureau, 'DAYS_CREDIT', ty, 1000)
  app_train_b['b_now_overdue_avg_' + ty] = mean_apps(index, bureau[(bureau['CREDIT_ACTIVE'] == 'Active') & (bureau['CREDIT_DAY_OVERDUE'] > 0)], 'CREDIT_DAY_OVERDUE', ty, 1000)
  app_train_b['b_closed_days_avg_' + ty] = mean_apps(index, bureau[bureau['DAYS_ENDDATE_FACT'] <= 0], 'DAYS_ENDDATE_FACT', ty, 1000)
  app_train_b['b_prolong_avg_' + ty] = mean_apps(index, bureau[bureau['CNT_CREDIT_PROLONG'] > 0], 'CNT_CREDIT_PROLONG', ty, 1000)
  app_train_b['b_amt_annuity_' + ty] = mean_apps(index, bureau[bureau['AMT_ANNUITY'] > 0], 'AMT_ANNUITY', ty, 1000)
  app_train_b['b_debt_avg_' + ty] = mean_apps(index, bureau[bureau['AMT_CREDIT_SUM_DEBT'] > 0], 'AMT_CREDIT_SUM_DEBT', ty, 1000)
  if ty != 'Credit card':
    app_train_b['b_dur_avg_' + ty] = mean_apps(index, bureau[bureau['CREDIT_ACTIVE'] == 'Closed'], 'fact_credit_duration', ty, 1000)

app_train_b['b_limit_avg_'] = mean_apps(index, bureau[bureau['AMT_CREDIT_SUM_LIMIT'] > 0], 'AMT_CREDIT_SUM_LIMIT', 'Credit card', 1000)
app_train_b['b_closed_percent_avg'] = mean_apps(index, bureau[bureau['closed_percent'].isna() == False], 'closed_percent', 'all', 1000)
app_train_b['b_dpd_avg'] = mean_apps(index, bureau[bureau['dpd'].isna() == False], 'dpd', 'all', 1000)
app_train_b['b_long_percent_avg'] = mean_apps(index, bureau[bureau['long_percent'].isna() == False], 'long_percent', 'all', 1000)
app_train_b['b_good_percent_avg'] = mean_apps(index, bureau[bureau['good_percent'].isna() == False], 'good_percent', 'all', 1000)

In [ ]:
#max
for t in time:
  for ty in types:
    if t != 1000:
      app_train_b['b_overdue_max_' + ty+'_'+str(t)] = max_apps(index, bureau[bureau['AMT_CREDIT_SUM_OVERDUE'] > 0], 'AMT_CREDIT_SUM_OVERDUE', ty, t)
    else:
      app_train_b['b_overdue_max_' + ty] = max_apps(index, bureau[bureau['AMT_CREDIT_SUM_OVERDUE'] > 0], 'AMT_CREDIT_SUM_OVERDUE', ty, t)


for ty in types:
  app_train_b['b_' + ty + '_mean'] = max_apps(index, bureau, 'AMT_CREDIT_SUM', ty, 1000)
  app_train_b['b_active_' + ty + '_dur_max'] = max_apps(index, bureau[bureau['DAYS_CREDIT_ENDDATE'] > 0], 'DAYS_CREDIT_ENDDATE', ty, 1000)
  app_train_b['b_last_closed_' + ty] = bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].max()[app_train_b[app_train_b['b_active_all_num'] == 0].index]

app_train_b['b_limit_max'] = max_apps(index, bureau[bureau['AMT_CREDIT_SUM_LIMIT'] > 0], 'AMT_CREDIT_SUM_LIMIT', 'Credit card', 1000)
app_train_b['b_debt_max'] = max_apps(index, bureau[bureau['AMT_CREDIT_SUM_DEBT'] > 0], 'AMT_CREDIT_SUM_DEBT', 'all', 1000)

In [ ]:
#other
app_train_b['b_Microloan'] = (app_train_b['b_Microloan_num']/app_train_b['b_Microloan_num']).fillna(0)
for ty in types:
  app_train_b['b_currency_1_percent_'+ty] = (bureau[(bureau['CREDIT_CURRENCY'] == 'currency 1') & (bureau['CREDIT_TYPE'] == ty)].groupby('SK_ID_CURR')['CREDIT_CURRENCY'].count()/app_train_b['b_'+ty+'_num']).fillna(-1)
  app_train_b['b_now_overdue_percent_' + ty] = (app_train_b['b_now_overdue_num_' + ty] / app_train_b['b_active_'+ty+'_num']).fillna(-1)

for t in time:
  for ty in types:
    if t != 1000:
      app_train_b['b_overdue_'+ty+'_'+str(t)] = (app_train_b['b_overdue_num_'+ty+'_'+str(t)] / app_train_b['b_overdue_num_'+ty+'_'+str(t)]).fillna(0)
    else:
      app_train_b['b_overdue_'+ty] = (app_train_b['b_overdue_num_'+ty] / app_train_b['b_overdue_num_'+ty]).fillna(0)

min = pd.DataFrame(index = index, columns = ['0'])
min['0'] = bureau.loc[:, ['SK_ID_CURR', 'AMT_CREDIT_SUM_LIMIT']][bureau['CREDIT_TYPE'] == 'Credit card'].groupby('SK_ID_CURR').min()['AMT_CREDIT_SUM_LIMIT']
app_train_b['b_limit_min'] = min['0'].fillna(0)

In [ ]:
for c in app_train_b.columns[122:]:
  if app_train_b[c].sum() == 0:
   print(c)

In [ ]:
app_train_b

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,b_active_Consumer credit_dur_max,b_last_closed_Consumer credit,b_Mortgage_mean,b_active_Mortgage_dur_max,b_last_closed_Mortgage,b_Car loan_mean,b_active_Car loan_dur_max,b_last_closed_Car loan,b_all_mean,b_active_all_dur_max,b_last_closed_all,b_limit_max,b_debt_max,b_Microloan,b_currency_1_percent_Credit card,b_now_overdue_percent_Credit card,b_currency_1_percent_Consumer credit,b_now_overdue_percent_Consumer credit,b_currency_1_percent_Mortgage,b_now_overdue_percent_Mortgage,b_currency_1_percent_Car loan,b_now_overdue_percent_Car loan,b_currency_1_percent_all,b_now_overdue_percent_all,b_overdue_Credit card_1,b_overdue_Consumer credit_1,b_overdue_Mortgage_1,b_overdue_Car loan_1,b_overdue_all_1,b_overdue_Credit card_3,b_overdue_Consumer credit_3,b_overdue_Mortgage_3,b_overdue_Car loan_3,b_overdue_all_3,b_overdue_Credit card,b_overdue_Consumer credit,b_overdue_Mortgage,b_overdue_Car loan,b_overdue_all,b_limit_min
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,...,780.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,450000.0,780.0,NaN,31988.565,245781.0,0.0,1.0,0.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31988.565
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,810000.0,1216.0,NaN,810000.000,0.0,0.0,1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,...,0.0,-382.0,0.0,0.0,-382.0,0.0,0.0,-382.0,94537.8,0.0,-382.0,0.000,0.0,0.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,Religion,...,0.0,-783.0,0.0,0.0,-783.0,0.0,0.0,-783.0,146250.0,0.0,-783.0,0.000,0.0,0.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,0.035792,-16941,-1588,-4970.0,-477,NaN,1,1,1,1,1,0,Laborers,2.0,2,2,WEDNESDAY,16,0,0,0,0,0,0,Other,...,471.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,267606.0,471.0,NaN,0.000,240057.0,0.0,-1.0,-1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...

Признаки из previous_application

In [ ]:
prev = pd.read_csv(path + '/previous_application.csv')

In [ ]:
prev

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.000000,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.290,267295.5,311400.0,0.0,267295.5,WEDNESDAY,12,Y,1,0.000000,NaN,NaN,XAP,Approved,-544,Cash through the bank,XAP,NaN,Refreshed,Furniture,POS,XNA,Stone,43,Furniture,30.0,low_normal,POS industry with interest,365243.0,-508.0,362.0,-358.0,-351.0,0.0
1670210,2357031,334635,Consumer loans,6622.020,87750.0,64291.5,29250.0,87750.0,TUESDAY,15,Y,1,0.340554,NaN,NaN,XAP,Approved,-1694,Cash through the bank,XAP,Unaccompanied,New,Furniture,POS,XNA,Stone,43,Furniture,12.0,middle,POS industry with interest,365243.0,-1604.0,-1274.0,-1304.0,-1297.0,0.0
1670211,2659632,249544,Consumer loans,11520.855,105237.0,102523.5,10525.5,105237.0,MONDAY,12,Y,1,0.101401,NaN,NaN,XAP,Approved,-1488,Cash through the bank,XAP,"Spouse, partner",Repeater,Consumer Electronics,POS,XNA,Country-wide,1370,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-1457.0,-1187.0,-1187.0,-1181.0,0.0
1670212,2785582,400317,Cash loans,18821.520,180000.0,191880.0,NaN,180000.0,WEDNESDAY,9,Y,1,NaN,NaN,NaN,XNA,Approved,-1185,Cash through the bank,XAP,Family,Repeater,XNA,Cash,x-sell,AP+ (Cash loan),-1,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-1155.0,-825.0,-825.0,-817.0,1.0


In [ ]:
prev_id = pd.DataFrame(columns = ['SK_ID_CURR'])
prev_id['SK_ID_CURR'] = prev['SK_ID_CURR'].unique()
app_train_p = pd.DataFrame(app_train['SK_ID_CURR']).merge(prev_id, on='SK_ID_CURR').set_index(['SK_ID_CURR'])
app_train_p

""
SK_ID_CURR
100002
100003
100004
100006
100007
...
456251
456252
456253


In [ ]:
prev[prev['NAME_CONTRACT_STATUS'] == 'Refused']['NAME_CONTRACT_STATUS'] = 'Canceled'

In [ ]:
def p_count_apps(index, data, column, credit_type, time):
  num = pd.DataFrame(index = index, columns=['0'])
  if credit_type == 'all':
    num['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_DECISION'] >= -365*time].groupby('SK_ID_CURR')[column].count()
  else:
    num['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_DECISION'] >= -365*time][data['NAME_CONTRACT_TYPE'] == credit_type].groupby('SK_ID_CURR')[column].count()
  return num['0'].fillna(0)

def p_sum_apps(index, data, column, credit_type, time):
  sum = pd.DataFrame(index = index, columns=['0'])
  if credit_type == 'all':
    sum['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_DECISION'] >= -365*time].groupby('SK_ID_CURR')[column].sum()
  else:
    sum['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_DECISION'] >= -365*time][data['NAME_CONTRACT_TYPE'] == credit_type].groupby('SK_ID_CURR')[column].sum()
  return sum['0'].fillna(0)

def p_mean_apps(index, data, column, credit_type, time):
  mean = pd.DataFrame(index = index, columns=['0'])
  if credit_type == 'all':
    mean['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_DECISION'] >= -365*time].groupby('SK_ID_CURR')[column].mean()
  else:
    mean['0'] = data.loc[:, ['SK_ID_CURR', column]][data['DAYS_DECISION'] >= -365*time][data['NAME_CONTRACT_TYPE'] == credit_type].groupby('SK_ID_CURR')[column].mean()
  return mean['0'].fillna(0)

def p_percent_apps(index, data, credit_type):
  return (p_count_apps(index, data, 'SK_ID_PREV', credit_type, 1000) / app_train_p['p_' + credit_type + '_num'].fillna(1)).fillna(0)

In [ ]:
#count

time = [1, 3, 1000]
types = ['Consumer loans', 'Cash loans', 'Revolving loans', 'all']
index = app_train_p.index
for t in time:
  for ty in types:
    if t != 1000:
      app_train_p['p_' + ty + '_num_' + str(t)] = p_count_apps(index, prev, 'SK_ID_PREV', ty, t)
      app_train_p['p_' + ty + '_canceled_num_' + str(t)] = p_count_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Canceled'], 'SK_ID_PREV', ty, t)
    else:
      app_train_p['p_' + ty + '_num'] = p_count_apps(index, prev, 'SK_ID_PREV', ty, t)
      app_train_p['p_' + ty + '_canceled_num'] = p_count_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Canceled'], 'SK_ID_PREV', ty, t)

app_train_p['p_unused_num'] = p_count_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Unused offer'], 'SK_ID_PREV', 'all', 1000)
    

In [ ]:
#sum

for t in time:
  if t != 1000:
    app_train_p['p_sum' + str(t)] = p_sum_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'], 'AMT_APPLICATION','all', t)
  else:
    app_train_p['p_sum'] = p_sum_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'], 'AMT_APPLICATION', 'all', t)



In [ ]:
#mean

for t in time:
  for ty in types:
    if t != 1000:
      app_train_p['p_' + ty + '_sum_app_avg_' + str(t)] = p_mean_apps(index, prev, 'AMT_APPLICATION', ty, t)
      app_train_p['p_' + ty + '_sum_avg_' + str(t)] = p_mean_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'], 'AMT_CREDIT', ty, t)
    else:
      app_train_p['p_' + ty + '_sum_app_avg'] = p_mean_apps(index, prev, 'AMT_APPLICATION', ty, t)
      app_train_p['p_' + ty + '_sum_avg_'] = p_mean_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'], 'AMT_CREDIT', ty, t)

for ty in types:
  app_train_p['p_' + ty + '_down_payment_avg_'] = p_mean_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'], 'RATE_DOWN_PAYMENT', ty, 1000)

app_train_p['p_cnt_avg'] = p_mean_apps(index, prev, 'CNT_PAYMENT', 'all', 1000)

In [ ]:
#percent

for ty in types:
  app_train_p['p_' + ty + '_low_percent'] = p_percent_apps(index, prev[prev['AMT_CREDIT'] < prev['AMT_APPLICATION']], ty)
  app_train_p['p_' + ty + '_high_percent'] = p_percent_apps(index, prev[prev['AMT_CREDIT'] > prev['AMT_APPLICATION']], ty)

days = prev['WEEKDAY_APPR_PROCESS_START'].unique()
for d in days:
  app_train_p['p_app_' + d + '_percent'] = p_percent_apps(index, prev[prev['WEEKDAY_APPR_PROCESS_START'] == d], 'all')

app_train_p['p_morning_percent'] = p_percent_apps(index, prev[prev['HOUR_APPR_PROCESS_START'] <= 14], 'all')
app_train_p['p_evening_percent'] = p_percent_apps(index, prev[prev['HOUR_APPR_PROCESS_START'] > 14], 'all')

pay_types = prev['NAME_PAYMENT_TYPE'].unique()
for t in pay_types:
  if t != 'XNA':
      app_train_p['p_' + t + '_percent'] = p_percent_apps(index, prev[prev['NAME_PAYMENT_TYPE'] == t], 'all')


suite_groups = {'Family':['Children', 'Family'], 'Partner':['Spouse, partner'], 'Unaccompanied':['Unaccompanied'], 'Other':['Group of people', 'Other_A', 'Other_B']}
for g in suite_groups:
  prev[g] = np.vectorize(lambda x: 1 if x in suite_groups[g] else 0)(prev['NAME_TYPE_SUITE'])
  app_train_p['p_type_suite_' + g + '_percent'] = p_percent_apps(index, prev[prev[g] == 1], 'all')
  prev.drop(columns=[g])

for t in prev['NAME_PRODUCT_TYPE'].unique()[1:]:
  app_train_p['p_prod_type_' + t] = p_percent_apps(index, prev[prev['NAME_PRODUCT_TYPE'] == t], 'all')

for t in prev['NAME_YIELD_GROUP'].unique()[:-1]:
  app_train_p['p_yield_' + t] = p_percent_apps(index, prev[prev['NAME_YIELD_GROUP'] == t], 'all')

for t in time:
  if t != 1000:
    app_train_p['p_ins_percent_' + str(t)] = (p_count_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'][prev['NFLAG_INSURED_ON_APPROVAL'] == 1], 'SK_ID_PREV', 'all', t) / p_count_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'], 'SK_ID_PREV', 'all', t).fillna(1)).fillna(0)
  else:
    app_train_p['p_ins_percent'] =  (p_count_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'][prev['NFLAG_INSURED_ON_APPROVAL'] == 1], 'SK_ID_PREV', 'all', t) / p_count_apps(index, prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'], 'SK_ID_PREV', 'all', t).fillna(1)).fillna(0)

groups = {'Card':['Card X-Sell', 'Card Street'], 'Cash':['Cash'], 'Cash X-Sell':['Cash X-Sell: low', 'Cash X-Sell: middle', 'Cash X-Sell: high'],
          'Cash Street':['Cash Street: low', 'Cash Street: middle', 'Cash Street: high'], 'POS mobile':['POS mobile with interest', 'POS mobile without interest'],
          'POS household':['POS household with interest', 'POS household without interest'], 'POS industry':['POS industry with interest', 'POS industry without interest'],
          'POS others':['POS other with interest', 'POS others without interest']}
for g in groups:
  prev[g] = np.vectorize(lambda x: 1 if x in groups[g] else 0)(prev['PRODUCT_COMBINATION'])
  app_train_p['p_prod_group_' + g + '_percent'] = p_percent_apps(index, prev[prev[g] == 1], 'all')
  prev.drop(columns=[g])

In [ ]:
#other

a = prev.loc[:, ['SK_ID_CURR', 'NFLAG_LAST_APPL_IN_DAY']].groupby('SK_ID_CURR').min()
a.iloc[:,0] = np.vectorize(lambda x: 1 if x == 0 else 0)(a)
app_train_p['p_is_several_a_day'] = a

app_train_p['p_cnt_max'] = prev.loc[:, ['SK_ID_CURR', 'CNT_PAYMENT']].groupby('SK_ID_CURR')['CNT_PAYMENT'].max()

app_train_p['p_ins_ind'] = prev[prev['NAME_CONTRACT_STATUS'] == 'Approved'].sort_values(by='DAYS_DECISION',ascending=False).drop_duplicates(subset='SK_ID_CURR').loc[:,['SK_ID_CURR', 'NFLAG_INSURED_ON_APPROVAL']].groupby('SK_ID_CURR')['NFLAG_INSURED_ON_APPROVAL'].max()

new_c = prev[prev['NAME_CLIENT_TYPE'] == 'New'].loc[:, ['SK_ID_CURR', 'DAYS_DECISION']].groupby('SK_ID_CURR').min()
new_c[1] = np.vectorize(lambda x: 1 if x >= -365 else 0)(new_c['DAYS_DECISION'])
new_c[3] = np.vectorize(lambda x: 1 if x >= -365*3 else 0)(new_c['DAYS_DECISION'])
app_train_p['p_new_client_1'] = new_c[1]
app_train_p['p_new_client_1'] = app_train_p['p_new_client_1'].fillna(0)
app_train_p['p_new_client_3'] = new_c[3]
app_train_p['p_new_client_3'] = app_train_p['p_new_client_3'].fillna(0)

#most_popular
for ch in prev['CHANNEL_TYPE'].unique():
  app_train_p[ch] = p_count_apps(index, prev[prev['CHANNEL_TYPE'] == ch], 'SK_ID_PREV', 'all', 1000)
app_train_p['p_most_popular_channel'] = prev['CHANNEL_TYPE'].unique()[0]
for ch in prev['CHANNEL_TYPE'].unique()[1:]:
  app_train_p['p_most_popular_channel'] = np.vectorize(lambda x: ch if app_train_p.loc[x, ch] > app_train_p.loc[x, app_train_p.loc[x, 'p_most_popular_channel']] else app_train_p.loc[x, 'p_most_popular_channel'])(app_train_p.index)
app_train_p = app_train_p.drop(columns=prev['CHANNEL_TYPE'].unique())

inds = list(prev['NAME_SELLER_INDUSTRY'].unique())
inds.remove('XNA')
for i in inds:
  app_train_p[i] = p_count_apps(index, prev[prev['NAME_SELLER_INDUSTRY'] == i], 'SK_ID_PREV', 'all', 1000)
app_train_p['p_most_popular_industry'] = 0
for i in inds:
  app_train_p['p_most_popular_industry'] = np.vectorize(lambda x: i if (app_train_p.loc[x, 'p_most_popular_industry'] == 0) or
                                                              (app_train_p.loc[x, i] > app_train_p.loc[x, app_train_p.loc[x, 'p_most_popular_industry']])
                                                              else app_train_p.loc[x, 'p_most_popular_industry'])(app_train_p.index)
app_train_p = app_train_p.drop(columns=inds)

In [ ]:
app_train_bureau_prev = app_train_p.merge(app_train_b, on='SK_ID_CURR')
app_train = app_train.set_index('SK_ID_CURR')
app_train_p = app_train_p.merge(app_train, on='SK_ID_CURR')

In [ ]:
app_train_prev = app_train_p.loc[list(set(app_train_p.index) - set(app_train_bureau_prev.index))]
app_train_bureau = app_train_b.loc[list(set(app_train_b.index) - set(app_train_bureau_prev.index))]
app_train_only = app_train.loc[list(set(app_train.index) - set(app_train_bureau_prev.index) - set(app_train_bureau.index) - set(app_train_prev.index))]

app_train_only - Заявки по которым нет дополнительных данных

app_train_bureau - Заявки по которым есть данные только в bureau

app_train_prev - Заявки по которым есть данные только в previous_application

app_train_bureau_prev - Заявки по которым есть данные и в bureau и в previous_applications

In [ ]:
app_train_only.shape[0]+app_train_prev.shape[0]+app_train_bureau.shape[0]+app_train_bureau_prev.shape[0]

307511

In [ ]:
app_train.shape[0]

307511

In [ ]:
app_train_only.to_csv('app_train_only.csv', index=True)

In [ ]:
app_train_res = pd.concat([app_train_only, app_train_bureau, app_train_prev, app_train_bureau_prev])

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(app_train_res, test_size = 0.15, stratify=(app_train_res['TARGET'].astype(str) + app_train_res['NAME_CONTRACT_TYPE']))

In [ ]:
train.to_csv('train.csv', index=True)
test.to_csv('test.csv', index=True)